<a href="https://colab.research.google.com/github/apeksha-agase/customer-care-chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./sample_data/intents.json').read()
intents = json.loads(data_file)

In [ ]:
nltk.download('punkt')
  
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
nltk.download('wordnet')
print (len(words), "original words", words)
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
475 original words ['welcome', 'Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'I', 'want', 'my', 'ticket', 'Download', 'my', 'ticket', 'Give', 'me', 'my', 'ticket', 'I', 'want', 'to', 'cancel', 'my', 'flight', 'ticket', 'cancel', 'my', 'flight', 'I', 'want', 'to', 'reschedule', 'my', 'flight', 'ticket', 'reschedule', 'my', 'flight', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'where', 'is', 'order', 'with', 'id', '431B67', '?', 'track', 'order', '562B78', 'Where', 'is', 'my', 'order', 'with', 'id', '561A24', '?', 'order', 'id', 

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
from tensorflow.keras.optimizers import SGD
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

In [ ]:
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
model = load_model('chatbot_model.h5')
import json
import random, pickle
import numpy as np
intents = json.loads(open('./sample_data/intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    #print("res:",res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    #print("results:",results)
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    #print("resList:",return_list)
    return return_list

In [ ]:
def getResponse(tag, intents_json):
    list_of_intents = intents_json['intents']
    result = "---"
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    tag = ints[0]['intent']
    res = getResponse(tag, intents)
    return res

In [ ]:
def myTestBot():
    incoming_msg = input()
    welcome_reply = chatbot_response("welcome")
    print("bot:",welcome_reply)
    classtag = predict_class(incoming_msg, model)
    tag = classtag[0]['intent']
    while(tag == 'greeting'):
        res = getResponse(tag, intents)
        print("bot:",res)
        incoming_msg = input()
        classtag = predict_class(incoming_msg, model)
        tag = classtag[0]['intent']
    if(tag == 'flightTicket'):
        print("bot: ticket process started")
        res = getResponse('authenticateEmail', intents)
        print("bot:",res)
        incoming_msg = input()
        #API Call
        res = getResponse('emailAuthSuccRes', intents)
        print("bot:",res)
        res = getResponse('authenticatePhone', intents)
        print("bot:",res)
        incoming_msg = input()
        #API Call
        res = getResponse('phoneAuthSuccRes', intents)
        print("bot:",res)
        #ApI Call to download ticket
        print("ticket url")
    elif(tag == 'flightCancel'):
        print("bot: cancellation process started")
        res = getResponse('authenticateEmail', intents)
        print("bot:",res)
        incoming_msg = input()
        #API Call
        res = getResponse('emailAuthSuccRes', intents)
        print("bot:",res)
        res = getResponse('authenticatePhone', intents)
        print("bot:",res)
        incoming_msg = input()
        #API Call
        res = getResponse('PhoneAuthSuccRes', intents)
        print("bot:",res)
        res = getResponse('enterPNR', intents)
        print("bot:",res)
        incoming_msg = input()
        #API Calls
        res = getResponse('cancelSuccRes', intents)
        print("bot:",res)
    else:
        res = getResponse('noanswer', intents)
        print("bot:",res)    

myTestBot()